### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn import metrics

from tqdm.auto import tqdm
import xgboost as xgb

### Data preparation and data clearning

**Some observation:**

According to the introduction of the competition, the data is interpreted as follows:
- `id`: Unique identifier for each data point.
- `age`: Age of the individual, categorized in 5-year intervals.
- `height(cm)`: Height of the individual in centimeters.
- `weight(kg)`: Weight of the individual in kilograms.
- `waist(cm)`: Waist circumference of the individual in centimeters.
- `eyesight(left/right)`: Eyesight measurements for the left and right eyes.
- `hearing(left/right)`: Hearing ability for the left and right ears, represented as binary.
- `systolic`: Systolic blood pressure measurement.
- `relaxation`: Diastolic blood pressure measurement.
- `fasting blood sugar`: Fasting blood sugar level.
- `Cholesterol`: Total cholesterol level.
- `triglyceride`: Triglyceride level.
- `HDL`: High-density lipoprotein cholesterol level.
- `LDL`: Low-density lipoprotein cholesterol level.
- `hemoglobin`: Hemoglobin level in the blood.
- `Urine protein`: Level of protein in urine, categorized.
- `serum creatinine`: Serum creatinine level.
- `AST`: Level of aspartate aminotransferase enzyme.
- `ALT`: Level of alanine aminotransferase enzyme.
- `Gtp`: Level of gamma-glutamyl transferase enzyme.
- `dental caries`: Presence (1) or absence (0) of dental cavities.
- `smoking`: Target variable indicating if the individual is a smoker (1) or not (0).

In [2]:
df = pd.read_csv('smoker_train_dataset.csv')
pg = pd.read_csv('train.csv') 
df.shape , pg.shape

((38984, 23), (159256, 24))

In [3]:
df = pd.concat([pg, df])
df.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0.0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1.0,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2.0,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3.0,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4.0,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [4]:
df.drop(columns=['id'], inplace=True)

In [5]:
def summary(df):
    sum = pd.DataFrame(df.dtypes, columns=['dtypes'])
    sum['missing #'] = df.isna().sum()
    sum['missing %'] = (df.isna().sum())/len(df)
    sum['uniques'] = df.nunique().values
    sum['count'] = df.count().values
    return sum
summary(df)

,dtypes,missing #,missing %,uniques,count
age,int64,0,0.0,18,198240
height(cm),int64,0,0.0,15,198240
weight(kg),int64,0,0.0,29,198240
waist(cm),float64,0,0.0,548,198240
eyesight(left),float64,0,0.0,20,198240
eyesight(right),float64,0,0.0,18,198240
hearing(left),int64,0,0.0,2,198240
hearing(right),int64,0,0.0,2,198240
systolic,int64,0,0.0,128,198240
relaxation,int64,0,0.0,94,198240


### EDA, feature importance analysis


In [6]:
df.columns

Index(['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)',
       'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic',
       'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride',
       'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST',
       'ALT', 'Gtp', 'dental caries', 'smoking'],
      dtype='object')

In [8]:
target = ['smoking']
discrete = ['age', 'height(cm)', 'weight(kg)', 'systolic', 'relaxation', 
           'fasting blood sugar', 'Cholesterol', 'triglyceride', 'HDL',
           'LDL', 'AST', 'ALT', 'Gtp']
continous = ['waist(cm)', 'eyesight(left)', 'eyesight(right)', 'hemoglobin',
             'serum creatinine']
binary = ['hearing(left)', 'hearing(right)', 'dental caries']
nominal = ['Urine protein']

num_var = discrete + continous


In [ ]:
# for var in num_var:
#     sns.histplot(data=df, x=var, kde=True)
#     plt.title(f"Distribution of {var}")
#     plt.show()   

In [ ]:
df[num_var].describe().T

In [ ]:
cat_var = target + binary + nominal

for var in cat_var:
    sns.countplot(data=df, x=var)
    plt.title(f"Count of {var}")
    plt.show()  

In [ ]:
selected_var = ['age', 'relaxation', 'systolic', 'hemoglobin', 'Gtp', 'ALT', 'AST', 'LDL', 'HDL', 'Cholesterol', 'eyesight(right)', 'eyesight(left)']
for var in selected_var:
    sns.kdeplot(data=df, x=var, hue=target[0])
    plt.title(f"Distribution of {var} for smokers v. non-smokers")
    plt.show()   

In [ ]:
corr_matrix = df[num_var].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(15, 12))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='Blues',  fmt='.2f', linewidths=2 )

plt.title('Correlation Matrix', fontsize=15)
plt.show()

Since the correlation values between any two variables is not greater than 90%, none of the attributes were removed.



In [9]:
df['bmi'] = df['weight(kg)'] / (df['height(cm)'] / 100) ** 2
features = list(df.columns)
features.pop(-2)

'smoking'

### Model selection process and parameter tuning

In [10]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train_full = df_train_full.smoking.values
y_train = df_train.smoking.values
y_val = df_val.smoking.values
y_test = df_test.smoking.values

del df_train['smoking']
del df_val['smoking']
del df_test['smoking']

In [11]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

def graph_roc(y,y_pred):
    fpr , tpr , thresholds = metrics.roc_curve(y, y_pred)
    plt.figure(figsize=(5, 5))
    plt.plot(fpr, tpr,label='Model')
    plt.plot([0, 1], [0, 1],label='Random', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.show()
    # print(metrics.auc(fpr, tpr))


def met(y,y_pred,above_average):
    print('Model Accuracy:',round(metrics.accuracy_score(y, above_average),4))
    print('ROC:',round(roc_auc_score(y, y_pred),4))
    print((metrics.confusion_matrix(y, above_average)/metrics.confusion_matrix(y, above_average).sum()).round(4))


#### Logistic Regression

In [ ]:
def train(df, y,  c=1.0):
    # dicts = df.to_dict(orient='records')
    # dv = DictVectorizer(sparse=False)
    # X = dv.fit_transform(dicts)
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=1)
    # model.fit(X, y)
    model.fit(df, y)      
    return dv, model

def predict(df,dv,model):
    # dicts = df.to_dict(orient='records')
    # X = dv.transform(dicts)
    # y_pred = model.predict_proba(X)[:, 1]
    y_pred = model.predict_proba(df)[:, 1]    
    return y_pred

dv, model = train(df_train, y_train)
y_pred = predict(df_val,dv,model)
above_average = (y_pred >= 0.5)
met(y_val,y_pred,above_average)
graph_roc(y_val,y_pred)

Determine best C for logistic regression

In [ ]:
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

for c in tqdm([0.01, 0.1, 1.0 , 0.5, 10]):

    scores = []

    for train_idx ,val_idx in kfold.split(df_train_full):
        df_train = df_train_full.iloc[train_idx]
        df_val = df_train_full.iloc[val_idx]

        y_train = df_train.smoking.values
        y_val = df_val.smoking.values
        
        del df_train['smoking']
        del df_val['smoking']
        
        dv,model = train(df_train,y_train,c)
        y_pred = predict(df_val,dv,model)

        roc_auc = metrics.roc_auc_score(y_val,y_pred)
        scores.append(roc_auc)

    print('C=%s %f +- %f' % (c,np.mean(scores),np.std(scores)))

Select C=1.0 as best tuning parameter for logistic regression

C=1.0 0.827558 +- 0.002522


#### Random Forest

In [12]:
rf = RandomForestClassifier(random_state=29)
rf.fit(df_train, y_train)

RandomForestClassifier(random_state=29)

In [13]:
y_pred = rf.predict_proba(df_val)[:, 1]
roc_auc_score(y_val, y_pred)

0.8605074183907785

In [16]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 29,
 'verbose': 0,
 'warm_start': False}

The untuned RF has an accuracy of 86.05% on the validation dataset.

In [17]:
param_grid = {
              'n_estimators' :[110, 120, 130, 140, 150,160,170,180,190, 200],
              'max_depth' : [10,15,20,25,30 ,50],
              'min_samples_leaf': [1, 3, 5, 10, 20],
             }

In [22]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=29), param_grid=param_grid, verbose=4, n_jobs=-1)


In [23]:
grid_search.fit(df_train, y_train)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


KeyboardInterrupt: 

In [ ]:
final_model = grid_search.best_estimator_
final_model

In [ ]:
final_model.fit(df_train, y_train)
y_pred = final_model.predict_proba(df_val)[:, 1]
roc_auc_score(y_val, y_pred)

In [ ]:
y_pred = final_model.predict_proba(df_train)[:, 1]
roc_auc_score(y_train, y_pred)

In [ ]:
# # n_estimator = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
# # n_estimator = [20,40,60,80,100,120,140,160,180,200]
# n_estimator = [140,160,180,200,220,240,260]

# # max_depth = [10, 15, 20, 25]
# max_depth = [15, 20, 25,30]

all_scores =[]
for m in max_depth:
    for n in n_estimator:
        random = RandomForestClassifier(n_estimators=n,max_depth=m,random_state=1,n_jobs=-1)
        random.fit(df_train, y_train)
        y_pred_rf = random.predict_proba(df_val)[:, 1] 
        all_scores.append((m,n,metrics.roc_auc_score(y_val, y_pred_rf )))

In [ ]:
df_all_scores = pd.DataFrame(all_scores, columns=['max_depth','n_estimator','roc'])
df_all_scores.round(4).sort_values(by='roc',ascending=False)

In [ ]:
df_all_scores_pivot = df_all_scores.pivot(index='max_depth', columns=['n_estimator'],values=['roc'])
df_all_scores_pivot.round(4)
plt.figure(figsize = (16,5))
sns.heatmap(df_all_scores_pivot,annot=True,fmt='.3f')

In [ ]:
rf = RandomForestClassifier(n_estimators=25,max_depth=180,random_state=1,n_jobs=-1)
rf.fit(df_train, y_train)
importances = rf.feature_importances_
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importances.sort_values(by='Importance', ascending=False)

#### XGboost

In [ ]:
dtrain = xgb.DMatrix(df_train, label= y_train , feature_names = features)
dval = xgb.DMatrix(df_val, label= y_val , feature_names = features)

watchlist = [(dtrain,'train'),(dval,'val')]

In [ ]:
for x in range(3,10):    
    xgb_params = {
        'eta': 0.2, 
        'max_depth': x,
        'min_child_weight': 1,
        
        'objective': 'binary:logistic',
        'nthread': 8,
        'eval_metric': 'auc',
        'seed': 1,
        'verbosity': 1,
    }
    xgb_model = xgb.train(xgb_params,dtrain,verbose_eval=5,evals=watchlist,num_boost_round=100)
    y_pred = xgb_model.predict(dval)
    print('max',x,"    roc:",metrics.roc_auc_score(y_val, y_pred))

picking best parameters for xgboost 

max_depth = 6

eta = 0.2

In [ ]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    'eval_metric': 'auc',
    'seed': 1,
    'verbosity': 1,
}
xgb_model = xgb.train(xgb_params,dtrain,verbose_eval=5,evals=watchlist,num_boost_round=100)
y_pred = xgb_model.predict(dval)
metrics.roc_auc_score(y_val, y_pred)

# After tuning All models picking random forest